In [ ]:
s=100
X_train = []
y_train = []
for folder in  os.listdir(trainpath +'TrainingSet') : 
    files = gb.glob(pathname= str( trainpath +'TrainingSet//' + folder + '/*.png'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        X_train.append(list(image_array))
        y_train.append(code[folder])



X_pred = []
s=100
files = gb.glob(pathname= str(testpath + 'ValidationSet/*.png'))
for file in files: 
    image = cv2.imread(file)
    image_array = cv2.resize(image , (s,s))
    X_pred.append(list(image_array))



X_train = np.array(X_train)
y_train = np.array(y_train)
X_pred_array = np.array(X_pred)


print(f'X_train shape  is {X_train.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'X_pred shape  is {X_pred_array.shape}')



KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(s,s,3)),
        keras.layers.Conv2D(150,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Conv2D(120,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(50,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(120,activation='relu') ,    
        keras.layers.Dense(100,activation='relu') ,    
        keras.layers.Dense(50,activation='relu') ,        
        keras.layers.Dropout(rate=0.5) ,            
        keras.layers.Dense(3,activation='softmax') ,    
        ])

KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
print('Model Details are : ')
print(KerasModel.summary())


epochs = 15

ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=16,verbose=1, callbacks=[learn_control, checkpoint])


ModelLoss, ModelAccuracy = KerasModel.evaluate(X_train, y_train)

print('Total Loss is {}'.format(ModelLoss))
print('Total Accuracy is {}'.format(ModelAccuracy ))



y_result = KerasModel.predict(X_pred_array)

print('Prediction Shape is {}'.format(y_result.shape))



import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import fnmatch 
import os 
from sklearn.metrics import confusion_matrix

image_path = 'Data\ValidationSet'
for filename in os.listdir(image_path):
    filename = os.path.basename(filename)
    Actual_list    = []
    Predicted_list = []       
    if any(map(filename.startswith, 'InSitu_')):
        Actual='Insitu'
            
    if any(map(filename.startswith, 'SOB_B_')):
        Actual='Benign'
            
    if any(map(filename.startswith, 'SOB_M_')):
        Actual='Invasive'

plt.figure(figsize=(20,40))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),306))) : 
    plt.subplot(34,9,n+1)
    plt.imshow(X_pred[i]) 
    plt.axis('off')
    Predicted= getcode(np.argmax(y_result[i]))
    plt.title('Predicted:  ' + Predicted +'\n' + 'Actual:' + Actual)
    Actual_list.append(Actual)
    Predicted_list.append(Predicted)

In [ ]:
def get_tested_predictions(Actual_list, Predicted_list):
    correct_preds = []
    incorrect_preds = []
    correct_count = 0
    incorrect_count = 0
    for ii in range(len(Actual_list)):
        label_idx = np.argmax(Actual_list[ii])
        prediction_idx = np.argmax(Predicted_list[ii])
        prediction = preds[ii][prediction_idx]
        if prediction_idx == label_idx:
            correct_preds.append(prediction)
            incorrect_preds.append(np.nan)
            correct_count = correct_count + 1
        if prediction_idx != label_idx:
            incorrect_preds.append(prediction)
            correct_preds.append(np.nan)
            incorrect_count = incorrect_count + 1
    return correct_preds, correct_count, incorrect_preds, incorrect_count

correct_preds, correct_count, incorrect_preds, incorrect_count = get_tested_predictions(Actual_list,Predicted_list)
print("Correct predictions: %d/%d" % (correct_count,len(Predicted_list)))
print("Incorrect predictions: %d/%d" % (incorrect_count,len(Predicted_list)))

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

y_trues = [np.argmax(ii) for ii in Actual_list]
y_preds = [np.argmax(ii) for ii in Predicted_list]


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cnf_matrix = confusion_matrix(y_trues, y_preds)
class_names=list(set(np.concatenate((y_trues,y_preds),axis=0)))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix, without normalization')
plt.show()
